# Manual Image Alignment Tools

This notebook demonstrates two GUI utilities for manual image alignment:

- image registration by defining and correlating two pointsets on the static and moving image
- manual fine tuning of the alignment by applying stepwise translations, rotations and scalings

The transformations are computed in the backend using `skimage.transform.estimate_transform` and `skimage.transform.warp`. This allows us to transform images using Euclidean, Similarity and Affine transformation matrices (among a few others), as well as warping images using various types of interpolation.

First we must import some preliminaries:

In [ ]:
import numpy as np
import panel as pn
pn.config.inline = True
# confing.inline tells panel to load the necessary javascript files locally and not from the internet
# This is necessary when runing on the GRE649 server as it has an incomplete internet connection
pn.extension()

Load the local libraries, and make the notebook interface fullwidth for extra rendering space:

In [ ]:
from libertem_ui.utils.notebook_tools import notebook_fullwidth, stop_nb
from align_panel import point_registration, fine_adjust, array_format
notebook_fullwidth()

Load the input images, however you want. Both must be bare numpy arrays though not necessarily of the same shape.

- `static` refers to the image we use as the reference
- `moving` is the image we align onto `static`

If one image is much larger than the other it is perhaps better to place the larger one as `moving`, as this means it will be downsampled rather than upsampled. In practice the final transformation object can be easily inverted via methods provided by `skimage`.

When loading with hyperspy this can be quite slow due to Hyperspy issues.

In [ ]:
# import hyperspy.api as hs
# static = hs.load('1_Stack_PS_diff_C_P_Align_Mean.dm3').data
# moving = hs.load('2_Stack_PS_diff_C_P_Align_Align_Mean_flipped.dm3').data
initial_points = None
shape = (600, 600)
static = np.random.random(size=shape)
moving = np.random.random(size=shape)

Inital points can be a Pandas DataFrame with the following 4 columns: `cx`, `cy` for points in the static image, `moving_cx`, `moving_cy` for corresponding points in the moving iamge. Throughout `y` refers to a vertical coordinate, with origin at the top, and `x` refers to the horizontal coordinate, origin on the left.

## Point-based registration

This UI displays the two images side-by-side and lets the user draw and edit points on each to define pointwise correspondences. The two images should remain in sync, i.e. a point drawn or deleted on the left will appear/disappear on the right, and vice versa. However, the position of a given point is independent in each image. The colors of each point corresponds between images.

Use the point edit tool in the figure toolbar to edit existing points (this appears as a group of three points and cursor symbol). Hold-Click and drag a point to move it, press backspace while dragging to delete it. In some cases double clicking on the figure with this tool will create a new poing, but this is sometimes unreliable.

To add points, use the free-add tool, which appears as a point with a dotted line around it and a plus symbol. Drag a freehand line on the image to place a new point near to that line, then edit the position as above.

When happy with the points, choose the transformation type (Affine is probably correct) and press `Run` to compute the transform. You can view the result by adjust the `Overlay alpha` slider which transitions between the static and transformed moving image.

The two toolboxes allow you to adjust the colormaps of each image.

In [ ]:
layout, transform_getter = point_registration(static, moving, initial_points=initial_points)
layout

The `layout` object is the GUI to display after the cell. The `transform_getter` object is a callable function which returns the state of the GUI when the last transformation was computed, as a dictionary containing the transformation object 

The following cell halts execution of the notebook. Press continue once the point registration has been carried out, or immediately to skip to fine adjustment without doing this step.

In [ ]:
stop_nb()

If a transform object has been defined by the point-based GUI, it is passed onto the `fine_adjust` GUI.

## Fine adjust alignment

This UI diplays the static image with the pre-transformed moving image overlaid. Each image has an independently controllable colormap and alpha (transparency).

To the right of the image are a set of controls for manually translating, rotating and scaling the image. Each set of controls has a stepsize input which can be used to change the magnitude of the transformation.

The `center-origin` checkbox is used to control the origin of rotation and scaling, when checked the image will rotate and scale about the center. When unchecked a circular cursor apperars on the image which can be moved to define the origin point to use. This requires the circular cursor tool to be selected in the figure toolbar.

Also on the figure toolbar is a tool for free draw translation of the moving image (it appears as a line connecting two points). When selected, dragging a line on the image figure will translate the start point to the end point. This can be used to quickly move the image to the correct location, or move a specific point to a new location if this is critical.

At the top of the UI pane are the image transparency controls, and at the bottom are the colormap controls.

In [ ]:
layout, fine_getter = fine_adjust(static, moving, initial_transform=transform_getter().get('transform', None))
layout

Again, the `layout` object is the GUI and the `fine_getter` object is a callable that returns a dictionary of the transformation defined within it. If an initial transform was supplied, the transformation object returned will already include this as part of the matrix.

In [ ]:
stop_nb()

In [ ]:
pn.pane.Markdown(array_format(fine_getter()['transform'].params))